In [15]:
import pandas as pd
import os
import tensorflow as tf
import numpy as np

from utilities.preprocessors import preprocess, assign_gender, map_value_to_index
from utilities.visualizers import view_rows
from utilities.loaders import load_file, load_meta_data, load_lookup_array
from models.fil_lstm_glove import load_lstm_model

from dotenv import load_dotenv
from pathlib import Path

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
profiles_dump = pd.read_csv('../documents/profiles_dump.csv', index_col=0)
profiles_dump

,conn_link,conn_name,gender,salutation,email,mobile_no,company_name,is_scraped
0,https://www.linkedin.com/in/mhelbert-paredes-c...,"Mhelbert Paredes, CHRA",NaN,NaN,NaN,0,NaN,False
1,https://www.linkedin.com/in/mvidanes/,Mari Anthony Vidanes,NaN,NaN,NaN,0,NaN,False
2,https://www.linkedin.com/in/krystle-tacpalan-a...,Krystle Tacpalan,NaN,NaN,NaN,0,NaN,False
3,https://www.linkedin.com/in/venze-visda-chra-a...,"Venze Visda, CHRA",NaN,NaN,NaN,0,NaN,False
4,https://www.linkedin.com/in/jsales365/,Carlito Jr Sales,NaN,NaN,NaN,0,NaN,False
...,...,...,...,...,...,...,...,...
1261,https://www.linkedin.com/in/jamie-rose-cleofas...,Jamie Rose Cleofas,NaN,NaN,NaN,0,NaN,NaN
1262,https://www.linkedin.com/in/jovit-frias-06a561...,Jovit Frias,NaN,NaN,NaN,0,NaN,NaN
1263,https://www.linkedin.com/in/danlopezprofessional/,Mark Daniel Lopez,NaN,NaN,NaN,0,NaN,NaN
1264,https://www.linkedin.com/in/daena-michaela-roy...,Daena Michaela Ceria,NaN,NaN,NaN,0,NaN,NaN


In [17]:
conn_info = pd.read_csv('../documents/conn_info.csv', index_col=0)
conn_info

,email,mobile_no,company_name,conn_link,conn_name,gender,salutation
0,paulinomarcus2914@gmail.com,0,Dyson · Full-time,https://www.linkedin.com/in/paulinomarcus/,Marcus Paulino,male,NaN
1,jhericksid15@gmail.com,09294383786 (Home),Cielo Talent · Full-time,https://www.linkedin.com/in/jhericksid/,Jherick Sid Celestial,male,NaN
2,EvanJake.Montano@libertymutual.com,+639260328119 (Mobile),Tata Consultancy Services · Full-time,https://www.linkedin.com/in/evanjakesorianomon...,Evan Jake Soriano Montano,male,NaN
3,capiorheg@gmail.com,0,JBW Managed Services and Consulting Corp · Ful...,https://www.linkedin.com/in/reggielee-capio-a4...,Reggielee Capio,male,NaN
4,johncarlotadeo18@gmail.com,+639432681245 (Mobile),Trojan Recruitment Group · Full-time,https://www.linkedin.com/in/john-carlo-tadeo-2...,John Carlo Tadeo,male,NaN
...,...,...,...,...,...,...,...
1065,NaN,0,transcosmos Asia Philippines Inc. · Full-time,https://www.linkedin.com/in/jamie-rose-cleofas...,Jamie Rose Cleofas,female,NaN
1066,jovitfrias@gmail.com,0,RCM Health Care Services · Full-time,https://www.linkedin.com/in/jovit-frias-06a561...,Jovit Frias,NaN,NaN
1067,danlopezatwork@gmail.com,0,Abacus Service Corporation · Full-time,https://www.linkedin.com/in/danlopezprofessional/,Mark Daniel Lopez,male,NaN
1068,dmrc_06@yahoo.com,09983696222 (Mobile),RingCentral · Full-time,https://www.linkedin.com/in/daena-michaela-roy...,Daena Michaela Ceria,female,NaN


In [18]:
corrected_df = profiles_dump.merge(conn_info, on='conn_link', how='left')
corrected_df

,conn_link,conn_name_x,gender_x,salutation_x,email_x,mobile_no_x,company_name_x,is_scraped,email_y,mobile_no_y,company_name_y,conn_name_y,gender_y,salutation_y
0,https://www.linkedin.com/in/mhelbert-paredes-c...,"Mhelbert Paredes, CHRA",NaN,NaN,NaN,0,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.linkedin.com/in/mvidanes/,Mari Anthony Vidanes,NaN,NaN,NaN,0,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.linkedin.com/in/krystle-tacpalan-a...,Krystle Tacpalan,NaN,NaN,NaN,0,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN
3,https://www.linkedin.com/in/venze-visda-chra-a...,"Venze Visda, CHRA",NaN,NaN,NaN,0,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN
4,https://www.linkedin.com/in/jsales365/,Carlito Jr Sales,NaN,NaN,NaN,0,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1261,https://www.linkedin.com/in/jamie-rose-cleofas...,Jamie Rose Cleofas,NaN,NaN,NaN,0,NaN,NaN,NaN,0,transcosmos Asia Philippines Inc. · Full-time,Jamie Rose Cleofas,female,NaN
1262,https://www.linkedin.com/in/jovit-frias-06a561...,Jovit Frias,NaN,NaN,NaN,0,NaN,NaN,jovitfrias@gmail.com,0,RCM Health Care Services · Full-time,Jovit Frias,NaN,NaN
1263,https://www.linkedin.com/in/danlopezprofessional/,Mark Daniel Lopez,NaN,NaN,NaN,0,NaN,NaN,danlopezatwork@gmail.com,0,Abacus Service Corporation · Full-time,Mark Daniel Lopez,male,NaN
1264,https://www.linkedin.com/in/daena-michaela-roy...,Daena Michaela Ceria,NaN,NaN,NaN,0,NaN,NaN,dmrc_06@yahoo.com,09983696222 (Mobile),RingCentral · Full-time,Daena Michaela Ceria,female,NaN


#### remove conn_name_x, gender_x, salutation_x, email_x, mobile_no_x, company_name_x, is_scraped

In [19]:
final_df = corrected_df.drop(columns=["conn_name_y", "gender_x", "salutation_x", "email_x", "mobile_no_x", "company_name_x", "is_scraped"])
final_df

,conn_link,conn_name_x,email_y,mobile_no_y,company_name_y,gender_y,salutation_y
0,https://www.linkedin.com/in/mhelbert-paredes-c...,"Mhelbert Paredes, CHRA",NaN,NaN,NaN,NaN,NaN
1,https://www.linkedin.com/in/mvidanes/,Mari Anthony Vidanes,NaN,NaN,NaN,NaN,NaN
2,https://www.linkedin.com/in/krystle-tacpalan-a...,Krystle Tacpalan,NaN,NaN,NaN,NaN,NaN
3,https://www.linkedin.com/in/venze-visda-chra-a...,"Venze Visda, CHRA",NaN,NaN,NaN,NaN,NaN
4,https://www.linkedin.com/in/jsales365/,Carlito Jr Sales,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
1261,https://www.linkedin.com/in/jamie-rose-cleofas...,Jamie Rose Cleofas,NaN,0,transcosmos Asia Philippines Inc. · Full-time,female,NaN
1262,https://www.linkedin.com/in/jovit-frias-06a561...,Jovit Frias,jovitfrias@gmail.com,0,RCM Health Care Services · Full-time,NaN,NaN
1263,https://www.linkedin.com/in/danlopezprofessional/,Mark Daniel Lopez,danlopezatwork@gmail.com,0,Abacus Service Corporation · Full-time,male,NaN
1264,https://www.linkedin.com/in/daena-michaela-roy...,Daena Michaela Ceria,dmrc_06@yahoo.com,09983696222 (Mobile),RingCentral · Full-time,female,NaN


In [20]:
final_df.rename(columns={'email_y': 'email', 'mobile_no_y': 'mobile_no', 'company_name_y': 'company_name', 'conn_name_y': 'conn_name', 'gender_y': 'gender', 'salutation_y': 'salutation'}, inplace=True)
final_df

,conn_link,conn_name_x,email,mobile_no,company_name,gender,salutation
0,https://www.linkedin.com/in/mhelbert-paredes-c...,"Mhelbert Paredes, CHRA",NaN,NaN,NaN,NaN,NaN
1,https://www.linkedin.com/in/mvidanes/,Mari Anthony Vidanes,NaN,NaN,NaN,NaN,NaN
2,https://www.linkedin.com/in/krystle-tacpalan-a...,Krystle Tacpalan,NaN,NaN,NaN,NaN,NaN
3,https://www.linkedin.com/in/venze-visda-chra-a...,"Venze Visda, CHRA",NaN,NaN,NaN,NaN,NaN
4,https://www.linkedin.com/in/jsales365/,Carlito Jr Sales,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
1261,https://www.linkedin.com/in/jamie-rose-cleofas...,Jamie Rose Cleofas,NaN,0,transcosmos Asia Philippines Inc. · Full-time,female,NaN
1262,https://www.linkedin.com/in/jovit-frias-06a561...,Jovit Frias,jovitfrias@gmail.com,0,RCM Health Care Services · Full-time,NaN,NaN
1263,https://www.linkedin.com/in/danlopezprofessional/,Mark Daniel Lopez,danlopezatwork@gmail.com,0,Abacus Service Corporation · Full-time,male,NaN
1264,https://www.linkedin.com/in/daena-michaela-roy...,Daena Michaela Ceria,dmrc_06@yahoo.com,09983696222 (Mobile),RingCentral · Full-time,female,NaN


#### saving this will be as if we collected new profiles 

In [22]:
final_df.to_csv('../documents/conn_info.csv')